### Init


In [ ]:
import sys
sys.path.append("../..")

import numpy as np
from emlopt import solvers, surrogates
from emlopt.utils import *
from emlopt.problem import build_problem
from experiments.problems.simple_functions import polynomial, build_rosenbrock, mccormick

import logging
def create_logger(name):
    test_logger = logging.getLogger(name)
    stream = logging.StreamHandler(sys.stdout)
    stream.setFormatter(logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s'))
    if not test_logger.handlers:
        test_logger.addHandler(stream)
    test_logger.setLevel(logging.DEBUG)
    test_logger.propagate = False
    return test_logger
logger = create_logger('ebm')


In [ ]:
import numpy as np
import tensorflow as tf
import tensorflow_addons as tfa
import tensorflow_probability as tfp
from emlopt.surrogates.base_surrogate import BaseSurrogate
from emlopt.utils import min_max_scale_in, min_max_scale_out
import matplotlib.pyplot as plt
import tempfile
import math

def build_regressor(input_shape, depth=4, width=20):
    mdl = tf.keras.Sequential()
    mdl.add(tf.keras.layers.Input(shape=(input_shape,), dtype='float32'))
    for i in range(depth):
        mdl.add(tf.keras.layers.Dense(width, activation='relu'))
    mdl.add(tf.keras.layers.Dense(2, activation='linear'))
    return mdl

def shuffle_dataset(dataset_x, dataset_y):
    dataset_y = np.expand_dims(dataset_y, axis=-1)
    concatenated = np.concatenate((dataset_x,dataset_y), axis=-1)
    np.random.shuffle(concatenated)
    return concatenated[:,:dataset_x.shape[1]], concatenated[:,-1]

### Loss function

In [ ]:
def logprob(x, mean, std):
    log_unnormalized = -0.5 * tf.math.squared_difference(x / std, mean / std)
    log_normalization = tf.constant(0.5 * np.log(2. * np.pi), dtype=np.float) + tf.math.log(std)
    return log_unnormalized - log_normalization
    

def normal_logprob_loss(batchX, batchY, keras_mdl):
    p = keras_mdl(batchX)
    pmean = p[:,0]
    plogstd = p[:,1]
    pstd = tf.math.exp(plogstd)
    y_logprob = logprob(batchY, pmean, pstd)
    return tf.math.reduce_mean(-y_logprob)


def mse_loss(batchX, batchY, keras_mdl):
    p = keras_mdl(batchX)
    pmean = p[:,0]
    plogstd = p[:,1]
    pstd = tf.math.exp(plogstd)
    l = tf.math.squared_difference(pmean, batchY)
    return tf.math.reduce_mean(l)


def gauss_density_centered(x, std):
    return tf.math.exp(-0.5*(x / std)**2) / (math.sqrt(2*math.pi)*std)

def partition_loss(batchX, batchY, keras_mdl):
    stds = [0.01, 0.1]
    loss = 0
    for std in stds:
        noise = tf.random.normal(batchX.shape, 0, std)
        q = batchX + noise
        q_hat = keras_mdl(q)
        q_hat_std = tf.clip_by_value(q_hat[:,1], clip_value_min=-math.log(10), clip_value_max=math.log(10))
        q_hat_std = tf.math.exp(q_hat_std)
        w = gauss_density_centered(noise, std)
        loss += -tf.math.reduce_mean(tf.math.log(q_hat_std/w))
    return loss / len(stds)


def custom_loss(batchX, batchY, keras_mdl):
    return (
        normal_logprob_loss(batchX, batchY, keras_mdl) + 
        mse_loss(batchX, batchY, keras_mdl) +  
        partition_loss(batchX, batchY, keras_mdl) 
    )


### Training loop and plot util


In [ ]:
class EBM(BaseSurrogate):

    def __init__(self, *args, **kwargs):
        super(EBM, self).__init__(*args, **kwargs)
        
    def fit_surrogate(self, x, y):
        bs = self.batch_size if self.batch_size else x.shape[0]

        y_lb = self.problem.y_lb
        y_ub = self.problem.y_ub

        norm_x = min_max_scale_in(x, np.array(self.problem.input_bounds))
        norm_y = (y - y_lb) / (y_ub - y_lb)

        # optimizer = tfa.optimizers.AdamW(
        #     weight_decay=self.weight_decay, learning_rate=self.lr)

        optimizer = tf.optimizers.Adam(learning_rate=self.lr)

        keras_mdl = build_regressor(self.problem.input_shape, self.depth, self.width)
                                                                                                       
        history = []
        for epoch in range(self.epochs):
            print("\nStart of epoch %d" % (epoch,))
            bs = min(norm_x.shape[0], self.batch_size)
            norm_x, norm_y = shuffle_dataset(norm_x, norm_y)

            for batch_idx in range(math.ceil(norm_x.shape[0]/bs)):
                if batch_idx == math.ceil(norm_x.shape[0]/bs)-1:
                    batchX = norm_x[batch_idx*bs:]
                    batchY = norm_y[batch_idx*bs:]
                else:
                    batchX = norm_x[batch_idx*bs:(batch_idx+1)*bs]
                    batchY = norm_y[batch_idx*bs:(batch_idx+1)*bs]
                with tf.GradientTape() as tape:
                    loss_value = custom_loss(batchX, batchY, keras_mdl)
                    grads = tape.gradient(loss_value, keras_mdl.trainable_weights)
                optimizer.apply_gradients(zip(grads, keras_mdl.trainable_weights))
                
            print("Training loss ", loss_value)
            history.append(loss_value)
            
        plt.plot(history)
        plt.show()
        
        return keras_mdl

    def plot_predictions(self, keras_mdl, samples_x, samples_y):
        y_ub = self.problem.y_ub
        y_lb = self.problem.y_lb
        if self.problem.input_shape <= 2:
            x, y = self.problem.get_grid(100)
            scaled_x = min_max_scale_in(x, np.array(self.problem.input_bounds))

            prob_pred = keras_mdl(scaled_x)

            pred = prob_pred[:,0].numpy().ravel()
            pred = pred * (y_ub - y_lb) + y_lb
            std_pred = tf.math.exp(prob_pred[:,1]).numpy().ravel()
            std_pred = std_pred * (y_ub - y_lb) 

        # 1D domain
        if self.problem.input_shape == 1:
            fig = plt.figure(figsize=(15, 10))
            plt.xlim(self.problem.input_bounds[0])
            x = np.squeeze(x)
            plt.plot(x, y, c="grey")
            plt.plot(x, pred)
            plt.fill_between(x, pred-std_pred, pred+std_pred, alpha=0.3, color='tab:blue', label='+/- std')
            plt.scatter(samples_x, samples_y, c="orange")
            plt.legend(["GT", "predicted mean", "predicted CI", "samples"], prop={'size': 14})
            plt.savefig(f'{tempfile.gettempdir()}/chart.png')
            if is_plot_visible(): plt.show()
            else: plt.close()

        # 2D domain
        elif self.problem.input_shape == 2:
            fig = plt.figure(figsize=(15, 10))
            ax = fig.add_subplot(111, projection='3d')
            ax.scatter(
                samples_x[:, 0], samples_x[:, 1], samples_y, color="orange")
            ax.scatter(x[:, 0], x[:, 1], y, alpha=0.15, color="lightgrey")
            ax.scatter(x[:, 0], x[:, 1], pred, alpha=0.3)
            ax.scatter(x[:, 0], x[:, 1], pred-std_pred, alpha=0.3, color="lightblue")
            ax.scatter(x[:, 0], x[:, 1], pred+std_pred, alpha=0.3, color="lightblue")
            ax.view_init(elev=15, azim=60)
            plt.legend(["samples", "GT", "predicted mean", "predicted CI"], prop={'size': 14})
            plt.savefig(f'{tempfile.gettempdir()}/chart.png')
            if is_plot_visible(): plt.show()
            else: plt.close()

        else:
            self.logger.debug("Plot not available for high dimensional domains.")
            self.logger.debug(f"X:\n{samples_x}\nY:\n{samples_y}")

            scaled_samples_x = min_max_scale_in(samples_x,np.array(self.problem.input_bounds))
            prob_pred = keras_mdl(scaled_samples_x)
            mu = min_max_restore_out(prob_pred[:,0].numpy().ravel(), samples_y)

            self.logger.debug(f"mu pred\n{mu}")
            self.logger.debug(f"diff\n{samples_y-mu}")

### Dataset: Hole in the middle

In [ ]:
set_seed()

def linear_constraint(backend, model, xvars):
    x=xvars[0]
    return [
        [x<= 0.7 , "center_dist"],
        [x>= 0.3 , "center_dist2"]
    ]

problem = build_problem("polynomial_1D", lambda x: math.sin(x*10)/x/10, ['real'], [[-1,1]])
dataset = problem.get_dataset(200, backend_type='cplex')
problem.y_lb = -1
problem.y_ub = 1

X, Y = [], []
for ii, aa in enumerate(dataset[0]):
    if (aa > -0.5 and aa < -0.3) or aa > 0.75:
        X.append(aa)
        Y.append(dataset[1][ii])
X = np.stack(X)
Y = np.stack(Y)

surrogate_cfg = {
    "epochs": 200,
    "learning_rate": 5e-3,
    "weight_decay": 0,
    "batch_size": 32,
    "depth": 3,
    "width": 40,
}

surrogate_model = EBM(problem, surrogate_cfg, logger)
mdl = surrogate_model.fit_surrogate(X,Y)

surrogate_model.plot_predictions(mdl, X, Y)

### Dataset: Sinc

In [ ]:
set_seed()

problem = build_problem("polynomial_1D", lambda x: -math.sin(x*10)/x/20, ['real'], [[-1,1]])
dataset = problem.get_dataset(200, backend_type='cplex')
problem.y_lb = -1
problem.y_ub = 1

surrogate_cfg = {
    "epochs": 200,
    "learning_rate": 5e-3,
    "weight_decay": 0,
    "batch_size": 32,
    "depth": 3,
    "width": 40,
}

surrogate_model = EBM(problem, surrogate_cfg, logger)
mdl = surrogate_model.fit_surrogate(*dataset)

surrogate_model.plot_predictions(mdl, *dataset)

### Dataset: Ackley 10D

In [ ]:
set_seed()

from experiments.problems.simple_functions import build_ackley

def constraint_scbo(backend, milp_model, xvars):
    r = 5
    acc1 = 0
    for xi in xvars:
        acc1 += xi*xi
    acc2 = 0
    for xi in xvars:
        acc2 += xi
    return [[acc1 <= r*r, "center_dist"], [acc2 <= 0, "cst2"]]

problem = build_problem("ackley_10d_cst", *build_ackley(10), constraint_scbo)
dataset = problem.get_dataset(30, backend_type='cplex')
problem.y_lb = 0
problem.y_ub = 1000

surrogate_cfg = {
    "epochs": 1000,
    "learning_rate": 5e-3,
    "weight_decay": 0,
    "batch_size": 64,
    "depth": 2,
    "width": 40,
}

surrogate_model = EBM(problem, surrogate_cfg, logger)
mdl = surrogate_model.fit_surrogate(*dataset)

### EML embedding


In [ ]:
from emlopt.solvers.base_milp import BaseMILP
from emlopt.emllib.backend import Backend, get_backend
from emlopt.emllib.net.reader.keras_reader import read_keras_sequential
from emlopt.emllib.net.process import fwd_bound_tighthening, ibr_bounds
from emlopt.emllib.net import embed
from emlopt.emllib.util import pwl_exp


class UCB_EBM(BaseMILP):

    def __init__(self, *args, **kwargs):
        super(UCB_EBM, self).__init__(*args, **kwargs)
        self.lambda_ucb = self.cfg['lambda_ucb']

    @timer
    def propagate_bound(self, parsed_model, timeout=30):
        backend = get_backend(self.cfg['backend'])
        bounds = np.array([[0,1]]*(self.problem.input_shape+1)) # input shape + y
        parsed_model.layer(0).update_lb(bounds[:,0])
        parsed_model.layer(0).update_ub(bounds[:,1])
        method = self.cfg.get('bound_propagation', 'both') # both is default
        if method == 'ibr':
            self.logger.debug(f"Using Interval Based Reasoning bound propagation")
            ibr_bounds(parsed_model)
        elif method == 'milp':
            self.logger.debug(f"Using MILP forward bound tighthening bound propagation")
            fwd_bound_tighthening(backend, parsed_model, timelimit=timeout)
        elif method == 'both':
            self.logger.debug(f"Using IBR and then MILP bound propagation")
            ibr_bounds(parsed_model)
            fwd_bound_tighthening(backend, parsed_model, timelimit=timeout)
        else:
            raise Exception('Invalid bound propagation method')
        return parsed_model

    def embed_model(self, bkd, milp_model, parsed_model, new_bounds=None, name=""):
        # bounds computed with propagate bounds method
        score_lb = parsed_model.layer(-1).lb()[0].item()
        score_ub = parsed_model.layer(-1).ub()[0].item()
        self.logger.debug(f"Computed bounds: lb: {score_lb} ub: {score_ub}")
        # x decision variables
        xvars = []
        norm_xvars = []
        bounds = self.problem.input_bounds if new_bounds is None else new_bounds
        for i,b in enumerate(bounds):
            if self.problem.input_type[i] == "int":
                xvars.append(bkd.var_int(milp_model, lb=b[0], ub=b[1], name=f"{name}_x{str(i)}"))
            else:
                xvars.append(bkd.var_cont(milp_model, lb=b[0], ub=b[1], name=f"{name}_x{str(i)}"))
            # NN scaled input
            norm_xvars.append(bkd.var_cont(milp_model, lb=0, ub=1, name=f"{name}_norm_x{str(i)}"))
            bkd.cst_eq(milp_model, norm_xvars[-1] * (b[1] - b[0]), xvars[-1] - b[0], f"{name}_cst_norm_x{str(i)}")
        # y variable
        yvar = bkd.var_cont(milp_model, lb=self.problem.y_lb, ub=self.problem.y_ub, name=f"{name}_y")
        norm_yvar = bkd.var_cont(milp_model, lb=0, ub=1, name=f"{name}_norm_y")
        bkd.cst_eq(milp_model, norm_yvar * (self.problem.y_ub - self.problem.y_lb), yvar - self.problem.y_lb, f"{name}_cst_norm_y")
        # score variable
        scorevar = bkd.var_cont(milp_model, lb=score_lb, ub=score_ub, name=f"{name}_score")
        norm_scorevar = bkd.var_cont(milp_model, lb=0, ub=1, name=f"{name}_norm_score")
        bkd.cst_eq(milp_model, norm_scorevar * (score_ub - score_lb), scorevar - score_lb, f"{name}_cst_norm_score")

        embed.encode(bkd, parsed_model, milp_model, norm_xvars + [norm_yvar], [scorevar], name)
        return xvars, norm_xvars, yvar, norm_yvar, scorevar, norm_scorevar

    def extract_solution(self, solution_vars, name, scaled=False):
        opt_x = np.zeros(self.problem.input_shape)
        for i in range(self.problem.input_shape):
            if scaled:
                opt_x[i] = solution_vars[f"{name}_norm_x"+str(i)]
            else:
                opt_x[i] = solution_vars[f"{name}_x"+str(i)]
        return opt_x

    def solve(self, keras_model, samples_x, samples_y):
        bkd = get_backend(self.cfg['backend'])
        milp_model = bkd.new_model()

        parsed_mdl = read_keras_sequential(keras_model)
        parsed_mdl, _ = self.propagate_bound(parsed_mdl, timeout=30, timer_logger=self.logger)

        # embed 2 times the nn
        xvars, norm_xvars, yvar, norm_yvar, scorevar, norm_scorevar = self.embed_model(bkd, milp_model, parsed_mdl, name='nn1')
        # xvars_2, norm_xvars_2, yvar_2, norm_yvar_2, scorevar_2, norm_scorevar_2 = self.embed_model(bkd, milp_model, parsed_mdl, name='nn2')
        
        # bound the X to be the same
        # for ii,xx in enumerate(xvars):
        #     bkd.cst_eq(milp_model, xvars[ii], xvars_2[ii], "same_X")

        # bound the y to be different
        # y_delta = bkd.var_cont(milp_model, lb=0, ub=1, name="y_explorer")
        # bkd.add_cst(milp_model, norm_yvar >= y_delta+norm_yvar_2, "y_delta")

        # bound the scores to be different
        # s_delta = bkd.var_cont(milp_model, lb=0, ub=1, name="s_explorer")
        # bkd.add_cst(milp_model, norm_scorevar >= s_delta+norm_scorevar_2, "s_delta")

        # PI
        # current_min = np.min(samples_y)
        # bkd.add_cst(milp_model, yvar <= current_min, "pi")
        # bkd.add_cst(milp_model, norm_scorevar >= 0.5, "pi")

        # objective
        objective = norm_scorevar - norm_yvar

        if self.problem.constraint_cb is not None:
            csts = self.problem.constraint_cb(bkd, milp_model, xvars)  #TODO: cst xvar2 too
            for pc in csts:
                bkd.add_cst(milp_model, *pc)

        bkd.set_obj(milp_model, 'max', objective)
        bkd.set_determinism(milp_model)
        if self.logger.level == logging.DEBUG:
            bkd.set_extensive_log(milp_model)
        solution = bkd.solve(milp_model, self.solver_timeout)

        if solution['status'] == 'infeasible':
            raise Exception("Not feasible")

        # self.logger.debug(f"Solution: {solution}")
        # decision_variables = self.extract_solution(solution['vars'], "nn1")
        main_variables = {
            "objective": solution['obj'],
            "x": solution['vars']['nn1_x0'],
            "norm_score1": solution['vars']['nn1_norm_score'],
            # "norm_score2": solution['vars']['nn2_norm_score'],
            "y1": solution['vars']['nn1_y'],
            # "y2": solution['vars']['nn2_y'],
            # "norm y delta": solution['vars']['y_explorer'],
            # "s delta": solution['vars']['s_explorer']
        }
        if self.solution_callback is not None:
            self.solution_callback(main_variables, solution)

        #return decision_variables


### Run solver

In [ ]:
cfg = {"backend": 'cplex', "lambda_ucb": 1, "solver_timeout": 30}

def cb(main_variables, all_variables):
    logger.debug(main_variables)

milp_model = UCB_EBM(problem, cfg, 1, logger)
milp_model.solution_callback = cb
milp_model.optimize_acquisition_function(mdl, *dataset, timer_logger=logger)